In [276]:
import PyPDF2
import os
import glob
import re
import ast


directory = r'C:\Users\Sergio\Desktop\TWS\tws-workspace\closeAI\Examples\content_generation\content'
all_content = []

# Get all PDF files in the directory
pdf_files = glob.glob(os.path.join(directory, '*.pdf'))



In [277]:
import openai

openai.api_key = ""

def get_completion_and_token_count(messages, 
                                 model="gpt-3.5-turbo-16k", 
                                 temperature=0.0, max_tokens=5000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )

    content = response.choices[0].message["content"]


    token_dict = {
        'prompt_tokens': response['usage']['prompt_tokens'],
        'completion_tokens': response['usage']['completion_tokens'],
        'total_tokens': response['usage']['total_tokens'],
    }

    return content, token_dict

In [278]:
def find_substring(results, substrings):
    # For each substring
    for sub in substrings:
        # Store indices where the substring is found
        found_indices = []

        # For each result
        for index, sub_results in results:
            # For each substring result
            for sub_res, found in sub_results:
                # If the current substring matches and it was found
                if sub_res == sub and found:
                    # Store the index
                    found_indices.append(index)
        
        # If the substring was found at least once
        if found_indices:
            # Return the substring and the indices where it was found
            return sub, found_indices

    # If none of the substrings were found
    return None, []

In [279]:
def get_main_topics(pdf_file):
    text_content = []

    # Open the pdf file
    with open(pdf_file, 'rb') as file:
        # Initialize a pdf file reader
        reader = PyPDF2.PdfReader(file)

        # Loop through each page
        for i in range(len(reader.pages)):
            # Get the text content of the page
            page = reader.pages[i]
            text = page.extract_text()

            # Add the text to the list
            text_content.append(text)

    substrings = ['contents', 'indice', 'index', 'índice', 'table of contents']

    text_content_first = text_content[:15]

    results = list(filter(lambda x: any(sub_res[1] for sub_res in x[1]), enumerate(map(lambda s: [(sub, s.lower().find(sub) != -1) for sub in substrings], text_content_first))))

    # Specify the substrings in the order you want to search for them
    substrings = ["table of contents", "indice", "index", "índice", "contents"]

    # Call the function with your results and substrings
    found_substring, found_indices = find_substring(results, substrings)

    index_location = found_indices[:1]
    index_location = int(index_location[0])

    pattern = re.compile(r"^\d+\. .+", re.MULTILINE)

    all_matching_substrings = []

    for i in range(index_location, index_location + 3):
        matching_substrings = pattern.findall(text_content[i])

        # Initialize a dictionary to store the topic name and page number
        matching_dict = {}

        for substring in matching_substrings:
            # Use a regular expression to separate the topic name and page number
            match = re.match(r'^(\d+)\.\s+(.+?)(?:\s+(\d+))?\s*$', substring)
            if match:
                # Store the topic name and page number in the dictionary
                matching_dict[match.group(2)] = match.group(3) if match.group(3) else None
        
        # Only append the dictionary if it is not empty
        if matching_dict:
            all_matching_substrings.append(matching_dict)

    print(all_matching_substrings)

    system_message_clean = f""" 
    You are going to receive an array of dictionaries. The keys are not cleaned and not clear. \n
    The array will be between triple backstrick (```). \n
    Output the same array of dictionaries with keys strings cleaned and its respective dictionary value. \n
    Just output the object asked for. ```{all_matching_substrings}``` """

    messages =  [  
    {'role':'system', 
    'content': system_message_clean},  
    ] 

    response, token_dict = get_completion_and_token_count(messages)
    print(response)
    print(token_dict)

    # Convert the string representation of the response into an actual list of dictionaries
    response = ast.literal_eval(response)

    all_content.append({"file_name": os.path.basename(pdf_file), "matching_substrings": response})


In [280]:
# Process all PDF files

for pdf_file in pdf_files:
    get_main_topics(pdf_file)

[{'Introduction to Human Nutrition: A Global Perspective on Food and Nutrition': '1', 'Body Composition': '12', 'Energy Metabolism': '31', 'Nutrition and Metabolism of Proteins and Amino Acids': '49', 'Digestion and Metabolism of Carbohydrates': '74', 'Nutrition and Metabolism of Lipids': '86', 'Dietary Reference Standards': '122', 'The Vitamins': '132', 'Minerals and Trace Elements': '188', 'Measuring Food Intake': '238', 'Food Composition': '276', 'Food and Nutrition: Policy and Regulatory Issues': '293', 'Nutrition Research Methodology': '305', 'Food Safety: A Public Health Issue of Growing Importance': '324', 'Food and Nutrition-Related Diseases: The Global Challenge': '350'}]
[{'Introduction to Human Nutrition: A Global Perspective on Food and Nutrition': '1', 'Body Composition': '12', 'Energy Metabolism': '31', 'Nutrition and Metabolism of Proteins and Amino Acids': '49', 'Digestion and Metabolism of Carbohydrates': '74', 'Nutrition and Metabolism of Lipids': '86', 'Dietary Refer

In [281]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(all_content)

[   {   'file_name': 'Introduction to Human Nutrition 2nd Edition ( PDFDrive '
                     ').pdf',
        'matching_substrings': [   {   'Body Composition': '12',
                                       'Dietary Reference Standards': '122',
                                       'Digestion and Metabolism of Carbohydrates': '74',
                                       'Energy Metabolism': '31',
                                       'Food Composition': '276',
                                       'Food Safety: A Public Health Issue of Growing Importance': '324',
                                       'Food and Nutrition-Related Diseases: The Global Challenge': '350',
                                       'Food and Nutrition: Policy and Regulatory Issues': '293',
                                       'Introduction to Human Nutrition: A Global Perspective on Food and Nutrition': '1',
                                       'Measuring Food Intake': '238',
                      